In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_nystrom as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.01
features = 1000
samples = 1000
calibrate = [False, True, True, True, True]
sample_split = [False, False, True, False, True]
marginal_loss = [False, False, False, False, False]
retrain_hypers = [False, False, False, True, True]

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

Task exception was never retrieved
future: <Task finished name='Task-294' coro=<Client._gather.<locals>.wait() done, defined at /nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-126' coro=<Client._gather.<locals>.wait() done, defined at /nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
    for i in range(len(calibrate)):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                              calibrate = calibrate[i],
                              sample_split = sample_split[i],
                              marginal_loss = marginal_loss[i],
                              retrain_hypers = retrain_hypers[i],
                              features = features, samples = samples)
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-0b86c4cd3761a688e71531c008391d62>,
 <Future: pending, key: main-be0b17215dc94c144e912fbdc855aa4f>,
 <Future: pending, key: main-db397a0f4c819f4da5fb91e07e9a9065>,
 <Future: pending, key: main-bae9ed1fd053b6b4ee8ead51d199c07e>,
 <Future: pending, key: main-66658fa58d2a7f33f025a0243452108a>,
 <Future: pending, key: main-c91b78a3ddf5f8ff192f51fb173ecb2a>,
 <Future: pending, key: main-c5534bf45da163f85db800eb32d2aeaa>,
 <Future: pending, key: main-15a6ee79fa8c9175551972e7e492c744>,
 <Future: pending, key: main-534bea7c0f022a4ade4518dafe98ca40>,
 <Future: pending, key: main-c2d4a0a4b5b05b8e3d8a9e77ae0d7d53>,
 <Future: pending, key: main-9897a7677f6e153beb7f5ff821d65750>,
 <Future: pending, key: main-ad079c8200c5e199034450b2dcd33675>,
 <Future: pending, key: main-4c05dc08caf482cc6bb4b51a5e1ee273>,
 <Future: pending, key: main-316b71e0efeb1b14e12478453a18c552>,
 <Future: pending, key: main-084132a1f18e1abde8d9468147b418c2>,
 <Future: pending, key: main-508a33a3335

In [6]:
# Getting results
results = client.gather(futures)

KeyboardInterrupt: 

In [7]:
# Closing client
client.close()
cluster.close()

In [ ]:
# Saving results
torch.save(obj = results,
           f = "ablation_nystrom_ntrial={0}_n={1}_d={2}_noise={3}.pt".format(ntrial,n,d,noise))

In [24]:
torch.tensor([[1],[2],[3]]).repeat_interleave(2).reshape(3,2)

tensor([[1, 1],
        [2, 2],
        [3, 3]])